In [1]:
import  numpy               as      np
import  torch
import  matplotlib.pyplot   as      plt

from    pathlib     import  Path
import  sys
sys.path.append(str(Path.cwd().parent.parent))
from    pytorch.torch_numerical             import  utils
from    pytorch.torch_numerical.solvers     import  FastSM_Boltzmann_VHS

dtype:  torch.dtype     = torch.float32
device: torch.device    = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
__dtype_str: str        = str(dtype).split('.')[-1]

dtype_and_device = {'dtype': dtype, 'device': device}

In [2]:
DIMENSION:  int     = 2
RESOLUTION: int     = 16
VHS_ALPHA:  float   = 0.0
file = np.load(f"../datasets__{__dtype_str}/Boltzmann__alpha_{int(VHS_ALPHA)}__res_{RESOLUTION}.npz")

V_MAX:      float   = float(file['v_max'])
DELTA_V:    float   = (2*V_MAX) / RESOLUTION
v_grid              = utils.velocity_grid(DIMENSION, RESOLUTION, V_MAX, **dtype_and_device)
FFT_AXES:   tuple[int]  = tuple(range(-(1+DIMENSION), -1))
FFT_NORM:   str     = 'forward'

VHS_COEFF:  float   = file['vhs_coeff']

In [3]:
solver_maxwellian = FastSM_Boltzmann_VHS(
    dimension   = DIMENSION,
    v_num_grid  = RESOLUTION,
    v_max       = V_MAX,
    vhs_alpha   = 0.0,
    vhs_coeff   = VHS_COEFF,
    
    quad_order_uniform  = 25,
    quad_order_legendre = 50,
    quad_order_lebedev  = 7,
    
    dtype   = dtype,
    device  = device,
)
solver_maxwellian.precompute()

solver_hard_sphere = FastSM_Boltzmann_VHS(
    dimension  = DIMENSION,
    v_num_grid  = RESOLUTION,
    v_max       = V_MAX,
    vhs_alpha   = 1.0,
    vhs_coeff   = VHS_COEFF,
    
    quad_order_uniform  = 25,
    quad_order_legendre = 50,
    quad_order_lebedev  = 7,
    
    dtype   = dtype,
    device  = device,
)
solver_hard_sphere.precompute()

In [4]:
__test_size = 16
__test_rand_indices = torch.randint(0, len(file['data']), size=(__test_size,))
err_order = 2.0
err_dim = tuple(range(1, 2+2*DIMENSION))

TEST__data_0 = torch.tensor(file['data'], **dtype_and_device)[__test_rand_indices]
TEST__gain_fft_0 = torch.tensor(file['gain_fft'], dtype=utils.dtype_real_to_complex(dtype), device=device)[__test_rand_indices]
TEST__loss_fft_0 = torch.tensor(file['loss_fft'], dtype=utils.dtype_real_to_complex(dtype), device=device)[__test_rand_indices]
TEST__collision_0: torch.Tensor = torch.fft.ifftn(TEST__gain_fft_0-TEST__loss_fft_0, dim=FFT_AXES, norm=FFT_NORM).real

TEST__gain_fft_0_pred = solver_maxwellian.compute_collision_gain_fft(None, torch.fft.fftn(TEST__data_0, dim=FFT_AXES, norm=FFT_NORM))
TEST__loss_fft_0_pred = solver_maxwellian.compute_collision_loss_fft(None, torch.fft.fftn(TEST__data_0, dim=FFT_AXES, norm=FFT_NORM))
TEST__collision_0_pred: torch.Tensor = torch.fft.ifftn(TEST__gain_fft_0_pred-TEST__loss_fft_0_pred, dim=FFT_AXES, norm=FFT_NORM).real

print(f"{TEST__collision_0.shape        = }")
print(f"{TEST__collision_0_pred.shape   = }")

TEST__collision_0.shape        = torch.Size([16, 1, 1, 16, 16, 1])
TEST__collision_0_pred.shape   = torch.Size([16, 1, 1, 16, 16, 1])


In [5]:
gain_fft_abs_error  = (TEST__gain_fft_0_pred - TEST__gain_fft_0).norm(p=err_order, dim=err_dim).mean().item()
loss_fft_abs_error  = (TEST__loss_fft_0_pred - TEST__loss_fft_0).norm(p=err_order, dim=err_dim).mean().item()
collision_abs_error = (TEST__collision_0_pred - TEST__collision_0).norm(p=err_order, dim=err_dim).mean().item()
print((f"Relative error (gain)      >>> {gain_fft_abs_error:.4e}"))
print((f"Relative error (loss)      >>> {loss_fft_abs_error:.4e}"))
print((f"Relative error (collision) >>> {collision_abs_error:.4e}"))

gain_fft_abs_error  = utils.absolute_error(TEST__gain_fft_0_pred,   TEST__gain_fft_0).mean().item()
loss_fft_abs_error  = utils.absolute_error(TEST__loss_fft_0_pred,   TEST__loss_fft_0).mean().item()
collision_abs_error = utils.absolute_error(TEST__collision_0_pred,  TEST__collision_0).mean().item()
print((f"Relative error (gain)      >>> {gain_fft_abs_error:.4e}"))
print((f"Relative error (loss)      >>> {loss_fft_abs_error:.4e}"))
print((f"Relative error (collision) >>> {collision_abs_error:.4e}"))

gain_fft_rel_error  = utils.relative_error(TEST__gain_fft_0_pred, TEST__gain_fft_0, p=err_order, dim=err_dim).mean().item()
loss_fft_rel_error  = utils.relative_error(TEST__loss_fft_0_pred, TEST__loss_fft_0, p=err_order, dim=err_dim).mean().item()
collision_rel_error = utils.relative_error(TEST__collision_0_pred, TEST__collision_0, p=err_order, dim=err_dim).mean().item()
print((f"Relative error (gain)      >>> {gain_fft_rel_error:.4e}"))
print((f"Relative error (loss)      >>> {loss_fft_rel_error:.4e}"))
print((f"Relative error (collision) >>> {collision_rel_error:.4e}"))

Relative error (gain)      >>> 0.0000e+00
Relative error (loss)      >>> 0.0000e+00
Relative error (collision) >>> 0.0000e+00
Relative error (gain)      >>> 0.0000e+00
Relative error (loss)      >>> 0.0000e+00
Relative error (collision) >>> 0.0000e+00
Relative error (gain)      >>> 0.0000e+00
Relative error (loss)      >>> 0.0000e+00
Relative error (collision) >>> 0.0000e+00


End of file